################ IMPORTING THE REQUIRED LIBRARIES

In [ ]:
import numpy as np
import soundfile
import os, glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import librosa
from sklearn import preprocessing 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

############## EMOTIONS INCLUDED IN THE DATASET

In [ ]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

############## EXTRACTING FEATURES FROM THE AUDIO SIGNAL USING LIBROSA

In [ ]:
def extract_feature(file_name, mfcc, chroma,spectral_centroid,spectral_bandwidth,
                    spectral_rolloff,spectral_contrast,rms,spectral_flatness):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
            
            
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))             
           
            
        if spectral_centroid:
            spectral_centroid=np.mean(librosa.feature.spectral_centroid(y=X, sr=sample_rate))
            result=np.hstack((result, spectral_centroid)) 
        
        if spectral_bandwidth:
           spectral_bandwidth=np.mean(librosa.feature.spectral_bandwidth(y=X, sr=sample_rate).T)
#           print(spectral_bandwidth)
           result=np.hstack((result, spectral_bandwidth)) 
           
        if spectral_rolloff:
           spectral_rolloff=np.mean(librosa.feature.spectral_rolloff(y=X, sr=sample_rate).T)
#           print(spectral_rolloff)
           result=np.hstack((result, spectral_rolloff))
        
        if spectral_contrast:
           spectral_contrast=np.mean(librosa.feature.spectral_contrast(y=X, sr=sample_rate))
           result=np.hstack((result, spectral_contrast))
           
        if rms:
           rms=np.mean(librosa.feature.rms(y=X).T,axis=0)
           result=np.hstack((result, rms))
           
        if spectral_flatness:
           spectral_flatness=np.mean(librosa.feature.spectral_flatness(y=X))
           result=np.hstack((result, spectral_flatness))
        
        return result

############## LOADING THE DATASET AND EXTRACTING ALL THE FEATURES FROM IT 

In [ ]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(r"F:\speech_project\speech-emotion-recognition-ravdess-data\Actor_*\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        feature=extract_feature(file, mfcc=True, chroma=True,spectral_centroid=True,spectral_bandwidth=True,
                                spectral_rolloff=True,spectral_contrast=True,rms=True,spectral_flatness=True)
        x.append(feature)
        y.append(emotion)
    return x,y


############## SHUFFLING OF OBS AND RESETTING THE OBS INDEXES

In [ ]:
X,y=load_data()
X=pd.DataFrame(X)
y=pd.DataFrame(y)
data=pd.concat([X,y],axis="columns")
data=data.sample(frac=1).reset_index(drop=True)
X=data.iloc[:,0:58]
y=data.iloc[:,58]

############## LABEL ENCODING THE RESPONSE VARIABLES

In [ ]:
le = preprocessing.LabelEncoder()
y=le.fit_transform(y)

############## SPLITTING THE DATA INTO TRAIN(95%) AND TEST(5%)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X, y, test_size=0.05, random_state=42)
print((x_train.shape[0], x_test.shape[0]))

print(f'Features extracted: {x_train.shape[1]}')

In [ ]:
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)

################### HYPER-PARAMETER TUNING

In [ ]:
model_svc=SVC(kernel='rbf',random_state=42)
param_svc={'gamma':['scale'],'decision_function_shape':['ovo']}

#################### TRAINING THE MODEL

In [ ]:
svmGrid = GridSearchCV(SVC(probability=True,kernel='linear'),param_grid=param_svc,
                       cv=kfold, scoring="roc_auc",verbose=3,n_jobs=-1)
svmGrid.fit(x_train, y_train)
y_pred_1=svmGrid.predict_proba(x_test)

print(svmGrid.best_params_)
print(svmGrid.best_score_)

print(accuracy_score(y_test, y_pred_1))

############# SAVING THE MODEL

In [ ]:
import pickle
pickle.dump(rfGrid, open("C:\\Users\\dbda\\Desktop\\project\\SVC.pkl", 'wb'))
print("Model Saved!!")

#################### PREDICTION ON UNSEEN DATA

In [ ]:
def load_test_data():
    x_test,y_test=[],[]
    
    file="F:\\speech_project\\UnSeenSet\\03-01-07-01-01-02-15.wav"
    emotion=emotions[file.split("-")[2]]
    feature=extract_feature(file, mfcc=True, chroma=True,spectral_centroid=True,spectral_bandwidth=True,
                            spectral_rolloff=True,spectral_contrast=True,rms=True,spectral_flatness=True)
    
    x_test.append(feature)
    y_test.append(emotion)
    print(x_test)
    return x_test,y_test

In [ ]:
X_test_data,y_test_data=load_test_data()
print(y_test_data)

########### LOADING SAVED MODEL AND PREDICTING

In [ ]:
import pickle
loaded_model = pickle.load(open("C:\\Users\\dbda\\Desktop\\project\\SVC.pkl", 'rb'))
result = loaded_model.predict(X_test_data)
print(result)

In [ ]:
print(le.inverse_transform(result))